In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
SCRIPT_DIR_PATH = os.getcwd()
OUTPUT_POSTPROCESSING_DIR_PATH = os.path.dirname(SCRIPT_DIR_PATH)
SSP_MODELING_DIR_PATH = os.path.dirname(OUTPUT_POSTPROCESSING_DIR_PATH)
SSP_OUTPUT_DIR_PATH = os.path.join(SSP_MODELING_DIR_PATH, "ssp_run_output")
CW_DATA_DIR_PATH = os.path.join(OUTPUT_POSTPROCESSING_DIR_PATH, "data")

In [4]:
ISO3 = "MAR"
REGION_NAME = "morocco"
RUN_DIR_PATH = os.path.join(SSP_OUTPUT_DIR_PATH, "sisepuede_results_sisepuede_run_2025-09-30T16;13;13.113881")

### Load emission targets and ssp outputs dfs

In [6]:
# Load emission targets
emission_targets_df = pd.read_csv(os.path.join(CW_DATA_DIR_PATH, "emission_targets_morocco_2022_LULUCF_update.csv"))
emission_targets_df.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR,Edgar_Class
0,agrc,CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,AG - Crops - CH4,0.026607,AG - Crops:CH4
1,agrc,CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,AG - Crops - CO2,0.611888,AG - Crops:CO2
2,agrc,N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,AG - Crops - N2O,4.621083,AG - Crops:N2O
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,4.577545,AG - Livestock:CH4
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,4.577545,AG - Livestock:CH4


In [7]:
# Load output data
ssp_output_df = pd.read_csv(os.path.join(RUN_DIR_PATH, 
                                         "sisepuede_results_sisepuede_run_2025-09-30T16;13;13.113881_WIDE_INPUTS_OUTPUTS.csv"))
ssp_output_df.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,morocco,0,0.0,4.609331e+06,4085.468561,1.190441e+06,28242.323335,152999.613918,950343.230957,...,12.059233,2.946800,6.177415,0.0,2.755621,8.636027,73.140598,37.547100,28.821448,92.81
1,0,morocco,1,0.0,4.560555e+06,4042.236066,1.177843e+06,27943.462606,151380.569494,940286.683299,...,12.539800,3.455483,7.165802,0.0,3.259699,8.228317,71.163825,39.000100,29.765171,92.81
2,0,morocco,2,0.0,4.672405e+06,4141.374219,1.206731e+06,28628.792023,155093.264619,963347.752475,...,12.161233,3.041817,11.119348,0.0,3.958685,8.302446,89.448975,39.119475,28.720595,92.81
3,0,morocco,3,0.0,4.759936e+06,4218.957338,1.229337e+06,29165.114227,157998.729910,981394.786726,...,12.289167,3.375283,5.436126,0.0,4.167037,8.846059,90.190264,37.352525,29.482348,92.81
4,0,morocco,4,0.0,4.709498e+06,4174.251507,1.216311e+06,28856.068519,156324.509507,970995.518476,...,12.337700,2.992183,6.177415,0.0,3.226093,7.882382,84.012849,38.551850,32.769776,92.81


### Obtain the ssp output values in the emission targets format

In [8]:
def sum_vars_from_ssp_outputs(
    emission_targets_df: pd.DataFrame,
    ssp_outputs_df: pd.DataFrame,
    vars_col: str = "Vars",
    out_col: str = "ssp_total",
    record_missing_col: str | None = "missing_vars",
    ssp_filter: dict | None = None,
) -> pd.DataFrame:
    """
    For each row in emission_targets_df, split the colon-separated strings in `vars_col`,
    find those columns in ssp_outputs_df, sum their values (over rows & columns), and
    write the total to `out_col` in emission_targets_df.

    Parameters
    ----------
    emission_targets_df : DataFrame
        Must contain a string column `vars_col` with colon-separated names.
    ssp_outputs_df : DataFrame
        Wide table whose columns include the names referenced by `emission_targets_df[vars_col]`.
    vars_col : str
        Column in emission_targets_df with colon-separated variable names.
    out_col : str
        New column to create in emission_targets_df with totals from ssp_outputs_df.
    record_missing_col : str | None
        If provided, creates a column listing any missing vars for each row.
    df2_filter : dict | None
        Optional filters to reduce ssp_outputs_df before summing, e.g.
        {"region": "egypt", "time_period": 7}

    Returns
    -------
    DataFrame
        emission_targets_df with new column `out_col` (and `record_missing_col` if requested).
    """
    # Optionally filter ssp_outputs_df by key=value pairs (e.g., region/time_period)
    if ssp_filter:
        mask = pd.Series(True, index=ssp_outputs_df.index)
        for k, v in ssp_filter.items():
            mask &= (ssp_outputs_df[k] == v)
        ssp_view = ssp_outputs_df.loc[mask]
    else:
        ssp_view = ssp_outputs_df

    # Ensure we only operate on numeric data when summing
    numeric_cols = set(ssp_view.select_dtypes(include=[np.number]).columns)

    def _total_for_vars(vars_str: str):
        if pd.isna(vars_str) or not str(vars_str).strip():
            return np.nan, []

        # Split, strip, and deduplicate while preserving order
        raw = [s.strip() for s in str(vars_str).split(":") if s.strip()]
        seen = set()
        cols = [c for c in raw if not (c in seen or seen.add(c))]

        present = [c for c in cols if c in ssp_view.columns and c in numeric_cols]
        missing = [c for c in cols if c not in ssp_view.columns or c not in numeric_cols]

        if not present or ssp_view.empty:
            return np.nan, missing

        # Sum over all filtered rows & all present columns
        vals = ssp_view[present].to_numpy(dtype=float, copy=False)
        total = np.nansum(vals)
        return float(total), missing

    totals, missings = [], []
    for v in emission_targets_df[vars_col].astype("string"):
        total, missing = _total_for_vars(v)
        totals.append(total)
        missings.append(missing)

    emission_targets_df = emission_targets_df.copy()
    emission_targets_df[out_col] = totals
    if record_missing_col is not None:
        emission_targets_df[record_missing_col] = missings

    return emission_targets_df


# -----------------------------
# Example usage
# -----------------------------

# If DF2 has a single row for the target (e.g., region="egypt", a specific time_period):
# df2_filter = {"region": "egypt"}              # or {"region": "egypt", "time_period": 7}
# If you want to sum across all rows of DF2, set df2_filter = None.

# df1_result = sum_vars_from_df2(DF1, DF2, vars_col="Vars",
#                                out_col="DF2_total",
#                                record_missing_col="Missing_in_DF2",
#                                df2_filter={"region": "egypt"})
# print(df1_result.head())


In [9]:
ssp_output_df.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,morocco,0,0.0,4.609331e+06,4085.468561,1.190441e+06,28242.323335,152999.613918,950343.230957,...,12.059233,2.946800,6.177415,0.0,2.755621,8.636027,73.140598,37.547100,28.821448,92.81
1,0,morocco,1,0.0,4.560555e+06,4042.236066,1.177843e+06,27943.462606,151380.569494,940286.683299,...,12.539800,3.455483,7.165802,0.0,3.259699,8.228317,71.163825,39.000100,29.765171,92.81
2,0,morocco,2,0.0,4.672405e+06,4141.374219,1.206731e+06,28628.792023,155093.264619,963347.752475,...,12.161233,3.041817,11.119348,0.0,3.958685,8.302446,89.448975,39.119475,28.720595,92.81
3,0,morocco,3,0.0,4.759936e+06,4218.957338,1.229337e+06,29165.114227,157998.729910,981394.786726,...,12.289167,3.375283,5.436126,0.0,4.167037,8.846059,90.190264,37.352525,29.482348,92.81
4,0,morocco,4,0.0,4.709498e+06,4174.251507,1.216311e+06,28856.068519,156324.509507,970995.518476,...,12.337700,2.992183,6.177415,0.0,3.226093,7.882382,84.012849,38.551850,32.769776,92.81


In [10]:
emission_targets_df_extended = sum_vars_from_ssp_outputs(emission_targets_df, ssp_output_df, vars_col="Vars",
                               out_col="ssp_emission",
                               record_missing_col="missing_in_ssp_outputs",
                               ssp_filter={"region": REGION_NAME, "primary_id": 0, "time_period": 7})

emission_targets_df_extended.head()

,Subsector,Gas,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,id,MAR,Edgar_Class,ssp_emission,missing_in_ssp_outputs
0,agrc,CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,AG - Crops - CH4,0.026607,AG - Crops:CH4,0.195274,[]
1,agrc,CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,AG - Crops - CO2,0.611888,AG - Crops:CO2,0.211910,[]
2,agrc,N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,AG - Crops - N2O,4.621083,AG - Crops:N2O,2.332593,[]
3,lvst,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,AG - Livestock - CH4,4.577545,AG - Livestock:CH4,1.073324,[]
4,lsmm,CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,AG - Livestock - CH4,4.577545,AG - Livestock:CH4,0.181252,[]


### Create diff report

In [11]:
# subset the emission targets to create the diff report template
diff_report_df = emission_targets_df_extended[[
    "Subsector",
    "id",
    ISO3,
    "ssp_emission",
]].copy()
diff_report_df.head()

,Subsector,id,MAR,ssp_emission
0,agrc,AG - Crops - CH4,0.026607,0.195274
1,agrc,AG - Crops - CO2,0.611888,0.211910
2,agrc,AG - Crops - N2O,4.621083,2.332593
3,lvst,AG - Livestock - CH4,4.577545,1.073324
4,lsmm,AG - Livestock - CH4,4.577545,0.181252


In [12]:
# merge subsector an id into a single column for clarity
diff_report_df["subsector_id"] = diff_report_df["Subsector"] + " - " + diff_report_df["id"]
diff_report_df = diff_report_df.drop(columns=["Subsector", "id"])
diff_report_df.head()

,MAR,ssp_emission,subsector_id
0,0.026607,0.195274,agrc - AG - Crops - CH4
1,0.611888,0.211910,agrc - AG - Crops - CO2
2,4.621083,2.332593,agrc - AG - Crops - N2O
3,4.577545,1.073324,lvst - AG - Livestock - CH4
4,4.577545,0.181252,lsmm - AG - Livestock - CH4


In [13]:
#rename region column
diff_report_df = diff_report_df.rename(columns={ISO3: "inventory_emission"})
diff_report_df.head()

,inventory_emission,ssp_emission,subsector_id
0,0.026607,0.195274,agrc - AG - Crops - CH4
1,0.611888,0.211910,agrc - AG - Crops - CO2
2,4.621083,2.332593,agrc - AG - Crops - N2O
3,4.577545,1.073324,lvst - AG - Livestock - CH4
4,4.577545,0.181252,lsmm - AG - Livestock - CH4


In [14]:
# Create inventory_share column
diff_report_df["inventory_share"] = diff_report_df["inventory_emission"] / diff_report_df["inventory_emission"].sum()
diff_report_df

,inventory_emission,ssp_emission,subsector_id,inventory_share
0,0.026607,0.195274,agrc - AG - Crops - CH4,0.000192
1,0.611888,0.211910,agrc - AG - Crops - CO2,0.004405
2,4.621083,2.332593,agrc - AG - Crops - N2O,0.033265
3,4.577545,1.073324,lvst - AG - Livestock - CH4,0.032951
4,4.577545,0.181252,lsmm - AG - Livestock - CH4,0.032951
5,0.061910,0.162365,lsmm - AG - Livestock - N2O,0.000446
6,0.000000,0.000000,ccsq - CCSQ - CH4,0.000000
7,0.000000,0.000000,ccsq - CCSQ - CO2,0.000000
8,0.000000,0.000000,ccsq - CCSQ - N2O,0.000000
9,0.415829,0.003007,scoe - EN - Building - CH4,0.002993


In [15]:
# Calculate error column, avoid division by zero by adding a small constant to the denominator
epsilon = 1e-8
diff_report_df["error"] = (diff_report_df["ssp_emission"] - diff_report_df["inventory_emission"]).abs() / (diff_report_df["inventory_emission"] + epsilon)
diff_report_df["squared_error"] = (diff_report_df["error"] ** 2)
diff_report_df.head()

,inventory_emission,ssp_emission,subsector_id,inventory_share,error,squared_error
0,0.026607,0.195274,agrc - AG - Crops - CH4,0.000192,6.339291,40.186616
1,0.611888,0.211910,agrc - AG - Crops - CO2,0.004405,0.653678,0.427295
2,4.621083,2.332593,agrc - AG - Crops - N2O,0.033265,0.495228,0.245251
3,4.577545,1.073324,lvst - AG - Livestock - CH4,0.032951,0.765524,0.586027
4,4.577545,0.181252,lsmm - AG - Livestock - CH4,0.032951,0.960404,0.922376


In [16]:
# Set subsector_id at the beginning of the df
diff_report_df = diff_report_df[[
    "subsector_id",
    "inventory_emission",
    "ssp_emission",
    "inventory_share",
    "error",
    "squared_error"
]]

# sort by squared_error descending
diff_report_df = diff_report_df.sort_values(by="squared_error", ascending=False)
diff_report_df.head(10)

,subsector_id,inventory_emission,ssp_emission,inventory_share,error,squared_error
37,frst - LULUCF - Forest Land Removals - CO2,0.000000,10.137287,0.000000,1.013729e+09,1.027646e+18
39,frst - LULUCF - HWP - CO2,0.000000,-2.426299,0.000000,2.426299e+08,5.886927e+16
42,lndu - LULUCF - Other Land - CO2,0.000000,-2.279446,0.000000,2.279446e+08,5.195876e+16
45,waso - Waste - Solid Waste - CO2,0.000000,2.265525,0.000000,2.265525e+08,5.132605e+16
33,ippu - IN - Industrial Processes - SF6,0.000000,0.280166,0.000000,2.801658e+07,7.849285e+14
32,ippu - IN - Industrial Processes - PFC,0.000000,0.056343,0.000000,5.634252e+06,3.174479e+13
31,ippu - IN - Industrial Processes - OTHER_FCS,0.000000,0.000013,0.000000,1.342789e+03,1.803083e+06
43,ippu - UNACCOUNTED - NF3,0.000000,0.000012,0.000000,1.217753e+03,1.482922e+06
16,fgtv - EN - Fugitive Emissions - CO2,0.014851,3.335980,0.000107,2.236305e+02,5.001059e+04
15,fgtv - EN - Fugitive Emissions - CH4,0.170801,3.777596,0.001229,2.111700e+01,4.459279e+02


In [17]:
diff_report_df.tail(40)

,subsector_id,inventory_emission,ssp_emission,inventory_share,error,squared_error
15,fgtv - EN - Fugitive Emissions - CH4,0.170801,3.777596,0.001229,2.111700e+01,4.459279e+02
38,frst - LULUCF - Forest Land Sequestration - CO2,-0.875000,-12.049629,-0.006299,-1.277100e+01,1.630986e+02
12,entc - EN - Electricity/Heat - CH4,0.006022,0.045412,0.000043,6.540715e+00,4.278095e+01
0,agrc - AG - Crops - CH4,0.026607,0.195274,0.000192,6.339291e+00,4.018662e+01
17,fgtv - EN - Fugitive Emissions - N2O,0.000734,0.005092,0.000005,5.937658e+00,3.525578e+01
26,trns - EN - Transportation - N2O,0.253538,0.827560,0.001825,2.264051e+00,5.125927e+00
18,inen - EN - Manufacturing/Construction - CH4,0.010545,0.033081,0.000076,2.137106e+00,4.567222e+00
20,inen - EN - Manufacturing/Construction - N2O,0.017397,0.051768,0.000125,1.975593e+00,3.902970e+00
24,trns - EN - Transportation - CH4,0.073646,0.210661,0.000530,1.860442e+00,3.461245e+00
5,lsmm - AG - Livestock - N2O,0.061910,0.162365,0.000446,1.622582e+00,2.632773e+00


### Save diff table

In [18]:
diff_report_df.to_csv(os.path.join(RUN_DIR_PATH, f"diff_report_{REGION_NAME}.csv"), index=False)